In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import layers
import pandas as pd
import os
import matplotlib.pyplot as plt
import copy
os.environ['CUDA_VISIBLE_DEVICES']="1"
pd_train=pd.read_csv("../../data/train.csv")
pd_test=pd.read_csv("../../data/test.csv")
production_year_max=0
production_year_min=0
model_release_year_max=0
model_release_year_min=0
distance_driven_max=0
distance_driven_min=0
Displacement_max=0
Displacement_min=0


In [ ]:
pd_train

In [ ]:
def TrainDatasetComposition(pddata,organize):
    brand=pd.unique(pddata["브랜드"])
    brand_record=np.copy(brand)
    sales_city=pd.unique(pddata["판매도시"])
    sales_city_record=np.copy(sales_city)
    sales_area=pd.unique(pddata["판매구역"])
    sales_area_record=np.copy(sales_area)
    vehicle_model=pd.unique(pddata["차량모델명"])
    vehicle_model_record=np.copy(vehicle_model)
    train_ID=np.array(pddata["ID"])
    pddata["생산년도"]=pddata["생산년도"]-2000
    pddata["모델출시년도"]=pddata["모델출시년도"]-2000
    pddata["주행거리"]=pddata["주행거리"]/10000
    
    if organize=="min":
        for i in range(len(brand)):
            brand_record[i]=pddata.loc[pddata["브랜드"]==brand[i]]["가격"].min()
        for i in range(len(sales_city)):
            sales_city_record[i]=pddata.loc[pddata["판매도시"]==sales_city[i]]["가격"].min()
        for i in range(len(sales_area)):
            sales_area_record[i]=pddata.loc[pddata["판매구역"]==sales_area[i]]["가격"].min()
        for i in range(len(vehicle_model)):
            vehicle_model_record[i]=pddata.loc[pddata["차량모델명"]==vehicle_model[i]]["가격"].min()
    elif organize=="mean":
        for i in range(len(brand)):
            brand_record[i]=pddata.loc[pddata["브랜드"]==brand[i]]["가격"].mean()
        for i in range(len(sales_city)):
            sales_city_record[i]=pddata.loc[pddata["판매도시"]==sales_city[i]]["가격"].mean()
        for i in range(len(sales_area)):
            sales_area_record[i]=pddata.loc[pddata["판매구역"]==sales_area[i]]["가격"].mean()
        for i in range(len(vehicle_model)):
            vehicle_model_record[i]=pddata.loc[pddata["차량모델명"]==vehicle_model[i]]["가격"].mean()
    elif organize=="max":
        for i in range(len(brand)):
            brand_record[i]=pddata.loc[pddata["브랜드"]==brand[i]]["가격"].max()
        for i in range(len(sales_city)):
            sales_city_record[i]=pddata.loc[pddata["판매도시"]==sales_city[i]]["가격"].max()
        for i in range(len(sales_area)):
            sales_area_record[i]=pddata.loc[pddata["판매구역"]==sales_area[i]]["가격"].max()
        for i in range(len(vehicle_model)):
            vehicle_model_record[i]=pddata.loc[pddata["차량모델명"]==vehicle_model[i]]["가격"].max()
    
    
    production_year_max=pddata["생산년도"].max()
    production_year_min=pddata["생산년도"].min()
    model_release_year_max=pddata["모델출시년도"].max()
    model_release_year_min=pddata["모델출시년도"].min()
    distance_driven_max=pddata["주행거리"].max()
    distance_driven_min=pddata["주행거리"].min()
    Displacement_max=pddata["배기량"].max()
    Displacement_min=pddata["배기량"].min()
 
    pddata["생산년도"]=(pddata["생산년도"]-production_year_min)/(production_year_max-production_year_min)
    pddata["모델출시년도"]=(pddata["모델출시년도"]-model_release_year_min)/(model_release_year_max-model_release_year_min)
    pddata["주행거리"]=(pddata["주행거리"]-distance_driven_min)/(distance_driven_max-distance_driven_min)
    pddata["배기량"]=(pddata["배기량"]-Displacement_min)/(Displacement_max-Displacement_min)

    pddata.insert(3,'생산-출시',pddata["생산년도"]-pddata["모델출시년도"])
    np_tarin=np.array(pddata)
    for i in range(len(np_tarin)):
        np_tarin[i,4]=brand_record[np.where(brand==np_tarin[i,4])[0][0]]/brand_record.max()
        np_tarin[i,5]=vehicle_model_record[np.where(vehicle_model==np_tarin[i,5])[0][0]]/vehicle_model_record.max()
        np_tarin[i,6]=sales_city_record[np.where(sales_city==np_tarin[i,6])[0][0]]/sales_city_record.max()
        np_tarin[i,7]=sales_area_record[np.where(sales_area==np_tarin[i,7])[0][0]]/sales_area_record.max()
    np_tarin=np.delete(np_tarin,0,axis=1)
    
    return np_tarin[:,:-1],np_tarin[:,-1]

In [5]:
pd_train1=copy.copy(pd_train)
mean_train_x,mean_train_y=TrainDatasetComposition(pd_train1,"mean")
mean_train_x=mean_train_x.astype(np.float32)

mean_train_y=mean_train_y.astype(np.float32)
pd_train1=copy.copy(pd_train)
min_train_x,min_train_y=TrainDatasetComposition(pd_train1,"min")
min_train_x=min_train_x.astype(np.float32)

min_train_y=min_train_y.astype(np.float32)
pd_train1=copy.copy(pd_train)
max_train_x,max_train_y=TrainDatasetComposition(pd_train1,"max")

max_train_x=max_train_x.astype(np.float32)
max_train_y=max_train_y.astype(np.float32)

In [6]:

def lstm_model():
    model = K.Sequential()
    model.add(layers.LSTM(56, input_shape = (14,1), return_sequences = True))
    model.add(layers.LSTM(28,  return_sequences = True))
    model.add(layers.LSTM(14,  return_sequences = False))
    model.add(layers.Dense(1))
    model.compile(loss = 'mean_absolute_error', optimizer = 'adam')
    return model

def gru_model():
    model = K.Sequential()
    model.add(layers.GRU(56, input_shape = (14,1), return_sequences = True))
    model.add(layers.GRU(28,  return_sequences = True))
    model.add(layers.GRU(14, return_sequences = False))
    model.add(layers.Dense(1))
    model.compile(loss = 'mean_absolute_error', optimizer = 'adam')
    return model

lstm_Model = lstm_model()
gru_Model = gru_model()

In [ ]:
mc = K.callbacks.ModelCheckpoint("../../model/gru/mean/bast_mse.h5", save_best_only=True)
gru_Model.fit(mean_train_x[:-10000],mean_train_y[:-10000],validation_data=(mean_train_x[-10000:],mean_train_y[-10000:]),callbacks=[mc], batch_size = 100, epochs = 2000, verbose = 1)
gru_Model.save("../../model/gru/mean/mse.h5")

In [7]:
mc = K.callbacks.ModelCheckpoint("../../model/lstm/mean/bast_mse.h5", save_best_only=True)
lstm_Model.fit(mean_train_x[:-10000],mean_train_y[:-10000],validation_data=(mean_train_x[-10000:],mean_train_y[-10000:]),callbacks=[mc], batch_size = 100, epochs = 2000, verbose = 1)
lstm_Model.save("../../model/lstm/mean/mse.h5")

Epoch 1/2000
480/480 [==============================] - 13s 16ms/step - loss: 43.6889 - val_loss: 39.6864
Epoch 2/2000
480/480 [==============================] - 7s 14ms/step - loss: 37.1279 - val_loss: 34.6447
Epoch 3/2000
480/480 [==============================] - 7s 14ms/step - loss: 33.0965 - val_loss: 31.5238
Epoch 4/2000
480/480 [==============================] - 7s 14ms/step - loss: 30.7148 - val_loss: 29.7706
Epoch 5/2000
480/480 [==============================] - 7s 14ms/step - loss: 29.3487 - val_loss: 28.7676
Epoch 6/2000
480/480 [==============================] - 7s 14ms/step - loss: 28.6210 - val_loss: 28.2717
Epoch 7/2000
480/480 [==============================] - 7s 14ms/step - loss: 28.2550 - val_loss: 28.0237
Epoch 8/2000
480/480 [==============================] - 7s 14ms/step - loss: 28.0921 - val_loss: 27.9294
Epoch 9/2000
480/480 [==============================] - 7s 14ms/step - loss: 28.0384 - val_loss: 27.8898
Epoch 10/2000
480/480 [==============================]

KeyboardInterrupt: 